In [1]:
# import module
from datetime import datetime
from datetime import timedelta
import requests
import os
import json
import Bearer
import pytz
import time


In [2]:
# 時刻を変換する関数

def make_local_dt (dt):
    utc_dt = dt - timedelta(hours=+9)
    return (pytz.utc.localize(utc_dt).astimezone(pytz.timezone("Asia/Tokyo")) )

    
def dt2localiso(dt):
    # この段階で
    if dt is None:
        return ''
    
    return dt.isoformat()

def iso_to_localdt(iso_str):
    dt = None
    try:
        dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%fZ')
        dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%f%z')
            dt = dt.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return dt

In [3]:
# 開始日
start_time_dt = make_local_dt(datetime(2021, 3, 26, 21, 0, 0) )# バイオハザード

# 終了日
end_time_dt = start_time_dt + timedelta(hours=+3)

start_time = dt2localiso(start_time_dt)
end_time = dt2localiso(end_time_dt)

In [4]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = Bearer.BEARER

search_url = "https://api.twitter.com/2/tweets/search/all"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print("response_st",response.status_code)
    if response.status_code != 200:
        raise Exception("response_st",response.status_code, response.text)
    return response.json()



In [25]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '#金曜ロードSHOW',
                'tweet.fields': "created_at,author_id,public_metrics", # スペースをいれない
                'max_results' : 500,
                "start_time":start_time ,
               "end_time": end_time,
               }

json_response = None
query_params.pop("until_id", None)
tweet_data = []

for i in range(10):
    time.sleep(1) # 連続するとエラーになる
    if json_response:
        print(i)
        query_params.pop("start_time", None)
        query_params.pop("end_time", None)
        query_params["until_id"] = int(json_response["data"][-1]["id"])
        
        print(query_params["until_id"])
    
    
    json_response = connect_to_endpoint(search_url, 
                                    create_headers(bearer_token), # headerを作る
                                    query_params,
                                    )
    for i,tmp_content in enumerate(json_response["data"]):
        
        
        try:
            tweet_data.append([tmp_content["id"], 
                               tmp_content["created_at"], tmp_content["text"].replace('\n',''),
                               tmp_content["public_metrics"]["like_count"],
                               tmp_content["public_metrics"]["retweet_count"], 
                               tmp_content["author_id"] ])
        except Exception as e:
            print(e)
        
        
        if i % 10 == 0:
            created = iso_to_localdt(tmp_content["created_at"])
            print(tmp_content["id"], created  )
            
        if created < start_time_dt :
            break
    
    if created < start_time_dt :
        break
    

response_st 200
1375462310957903876 2021-03-26 23:59:04+09:00
1375460575321366530 2021-03-26 23:52:10+09:00
1375459599612977158 2021-03-26 23:48:18+09:00
1375458081023254530 2021-03-26 23:42:15+09:00
1375457184885080066 2021-03-26 23:38:42+09:00
1375456795020324867 2021-03-26 23:37:09+09:00
1375456364059746310 2021-03-26 23:35:26+09:00
1375455723740532737 2021-03-26 23:32:53+09:00
1375455149578022916 2021-03-26 23:30:37+09:00
1375454557434572805 2021-03-26 23:28:15+09:00
1375454221080752131 2021-03-26 23:26:55+09:00
1375453811850928133 2021-03-26 23:25:18+09:00
1375453533844070403 2021-03-26 23:24:11+09:00
1375453335164100614 2021-03-26 23:23:24+09:00
1375452871391502343 2021-03-26 23:21:33+09:00
1375451999529918470 2021-03-26 23:18:06+09:00
1375451004271927296 2021-03-26 23:14:08+09:00
1375450326497566722 2021-03-26 23:11:27+09:00
1375449269620994052 2021-03-26 23:07:15+09:00
1375448651535806475 2021-03-26 23:04:47+09:00
1375448231815958535 2021-03-26 23:03:07+09:00
137544782800934092

In [53]:
import pandas as pd
df = pd.DataFrame(tweet_data, columns = ["tweet_id", "date","text","like","retweet", "author_id"])
df.set_index("tweet_id").to_csv("./0326baio.csv")

In [47]:
# !pip install pymlask

,date,text,like,retweet,author_id
tweet_id,,,,,
1375462310957903876,2021-03-26T14:59:04.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,220670839
1375461538350342144,2021-03-26T14:56:00.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,4114544294
1375461468389273600,2021-03-26T14:55:43.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,1283026644748271616
1375461362386698246,2021-03-26T14:55:18.000Z,RT @animatetimes: 『 #バイオハザード 』はいかがでしたか❓ 来週の #金...,0,47,1203834535349108736
1375461287375765511,2021-03-26T14:55:00.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,1277998462919835650
...,...,...,...,...,...
1375417399164342273,2021-03-26T12:00:36.000Z,今日は元祖サイコロステーキ先輩が見えるな…#金曜ロードSHOW https://t.co/N...,5,0,1194760349418844160
1375417388980559881,2021-03-26T12:00:34.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,1303191567478353920
1375417345712087046,2021-03-26T12:00:23.000Z,RT @Yukidaruma_4488: 今日の金曜ロードショーは、バイオハザード‼️元祖サ...,0,637,984274072002379776
